# <u> LSTM ENCODER DECODER

# Data

In [1]:
import utils

#utils.setup_nltk()
PRESIDENT = 'obama'
speeches = utils.read_all_text_files(PRESIDENT)

### Preprocessing

In [2]:
filter_list = [':', '(', ')', ',']
filtered_speeches = []

for speech in speeches:
    filtered_speech = []
    for word in speech:
        # filter out unwanted words
        if word not in filter_list:
            # lower word
            filtered_speech.append(word.lower())
    filtered_speeches.append(filtered_speech)

### Create N-Grams

In [3]:
from nltk import ngrams

WINDOW = 3
grams = [ngrams(s, WINDOW+1) for s in filtered_speeches]
flat_grams = [ng for speech in grams for ng in speech]

### Split Data

In [4]:
import pandas as pd

X = [' '.join(list(x[0:WINDOW])) for x in flat_grams]
Y = [x[-1] for x in flat_grams]
df = pd.DataFrame.from_dict({'x':X, 'y':Y})

# persist
csv_name = '../data/' + PRESIDENT + '/encdec_' + str(WINDOW) + 'grams.csv'
df.to_csv(csv_name, index=False)

In [5]:
import torchtext

XFIELD = torchtext.data.Field(sequential=True)
YFIELD = torchtext.data.Field(sequential=True)
DATA = torchtext.data.TabularDataset(csv_name,'csv', 
                                     [('x', XFIELD),('y', YFIELD)], skip_header=True)

XFIELD.build_vocab(DATA)  
YFIELD.build_vocab(DATA)

/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes f

In [6]:
from torchtext.data import BucketIterator, Iterator
import torch

BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator = Iterator(DATA, BATCH_SIZE, device=device, train=True)

/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: Iterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


# Neural Networks

### Encoder

In [7]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, embedding_dim, num_layers):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers)

    def forward(self, x, h0, c0):
        x = self.embedding(x).unsqueeze(0)
        out, (h0, c0) = self.lstm(x, (h0, c0))
        return out, (h0, c0)

### Decoder

In [8]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, embedding_dim, num_layers):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers)
        self.dense = nn.Linear(hidden_size, vocab_size)
        self.softmax = nn.LogSoftmax(dim = 1)
  
    def forward(self, x, h0, c0):
        x = self.embedding(x)
        x, (h0, c0) = self.lstm(x, (h0, c0))
        x = self.dense(x.squeeze(0))
        x = self.softmax(x)
        return x, (h0, c0)

### Training

In [31]:
HIDDEN_SIZE = 100
EMBEDDING_SIZE = 100
NUM_LAYERS = 3
ENC_LEARNING_RATE = 0.01
DEC_LEARNING_RATE = 0.01
criterion = nn.NLLLoss()

encoder = Encoder(len(XFIELD.vocab), HIDDEN_SIZE, EMBEDDING_SIZE, NUM_LAYERS).to(device)
decoder = Decoder(len(YFIELD.vocab), HIDDEN_SIZE, EMBEDDING_SIZE, NUM_LAYERS).to(device)
enc_optimizer = torch.optim.Adam(encoder.parameters(), lr = ENC_LEARNING_RATE)
dec_optimizer = torch.optim.Adam(decoder.parameters(), lr = DEC_LEARNING_RATE)

In [32]:
from tqdm import tqdm

EPOCHS = 10
for ep in range(EPOCHS):
    ep_loss = 0
    
    for batch in tqdm(train_iterator):
        if len(batch) != 64: break;
        inp = batch.x
        target = batch.y
        
        # init
        loss = 0
        h0 = torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).to(device)
        c0 = torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).to(device)
        enc_optimizer.zero_grad()
        dec_optimizer.zero_grad()
        
        # encode
        for w in range(inp.size(0)):
            enc_out, (h0, c0) = encoder(inp[w], h0, c0)
            
        # decode
        cur = inp[WINDOW-1].unsqueeze(0)
        dec_out, (h0, c0) = decoder(cur, h0, c0)
        cur = torch.ar
        
        gmax(dec_out,dim=1)

        # loss
        target_onehot = torch.nn.functional.one_hot(target.squeeze(), len(YFIELD.vocab))
        loss += criterion(dec_out, target.squeeze())
        
        # optimize
        ep_loss += loss
        loss.backward()
        enc_optimizer.step()
        dec_optimizer.step()
        
    print('AVG_LOSS={}, (ABS={})'.format(round((ep_loss/(len(DATA)/64)).item(),4), round(ep_loss.item(),2)))

  0%|          | 0/75665 [00:00<?, ?it/s]

AVG_LOSS=1.8733, (ABS=141743.62)


  0%|          | 0/75665 [00:00<?, ?it/s]

AVG_LOSS=1.5421, (ABS=116678.73)


  0%|          | 0/75665 [00:00<?, ?it/s]

...

AVG_LOSS=1.4417, (ABS=109085.17)


  0%|          | 0/75665 [00:00<?, ?it/s]

AVG_LOSS=1.4428, (ABS=109169.41)


100%|█████████▉| 75664/75665 [40:49<00:00, 30.89it/s]

AVG_LOSS=1.4429, (ABS=109173.57)


In [34]:
# SAVE ENCODER MODEL
ENC_PATH = "../data/obama/obama_lstm_enc.pt"
torch.save(encoder.state_dict(), ENC_PATH)

# SAVE ENCODER MODEL
DEC_PATH = "../data/obama/obama_lstm_dec.pt"
torch.save(decoder.state_dict(), DEC_PATH)

# Generate Text!

In [33]:
def voc_index(words):
    return torch.tensor([XFIELD.vocab.stoi[x] for x in words]).to(device)

def predict(inp):
    with torch.no_grad():

        h0 = torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).to(device)
        c0 = torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).to(device)
        
        for w in range(inp.size(0)):
                enc_out, (h0, c0) = encoder(inp[w], h0, c0)

        cur = inp[WINDOW-1].unsqueeze(0)
        dec_out, (h0, c0) = decoder(cur, h0, c0)
        cur = torch.argmax(dec_out,dim=1)
        
        # TODO pick more than first one
        return YFIELD.vocab.itos[cur[0].item()]

    
text = ['good', 'evening', 'ladies']
for i in range(20):
    cur_window = text[-WINDOW:]
    vecs = voc_index(cur_window).view(WINDOW,1).repeat(1,BATCH_SIZE)
    text.append(predict(vecs))

text

['good',
 'evening',
 'ladies',
 'good',
 'evening',
 'web',
 'good',
 'evening',
 'web',
 'good',
 'evening',
 'web',
 'good',
 'evening',
 'web',
 'good',
 'evening',
 'web',
 'good',
 'evening',
 'web',
 'good',
 'evening']